In [1]:
import tensorflow as tf
import numpy as np 
import torch
import random

/home/jszacha1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jszacha1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jszacha1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jszacha1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [25]:
def uniform_random(n, samples):
    indices = [0 for i in range(samples)]
    for i in range (samples):
        indis[i] = random.randint(int(i*2**(n*n)/samples), int((i+1)*2**(n*n)/samples))
    return indis


In [26]:
def indices ( num_indices , _range ):
    _list = [[]]
    return ( _index_gen_helper ( num_indices , _range , _list ))
def _index_gen_helper ( num_indices , _range , _list ):
    _ret = []

    for i in _list :
        for j in range ( _range ):
            _temp = i. copy ()
            _temp . extend ([ j ])
            _ret . append ( _temp )
    if num_indices == 1:
        return _ret
    else :
        return _index_gen_helper ( num_indices -1 , _range , _ret )

In [52]:
def instant_indices(num_independent_indices, i):
    inds=[0 for i in range (num_independent_indices)]
    for j in range (0, num_independent_indices):
        inds[j] = i%2
        i//=2
    return inds

print(instant_indices(5,20))

[0, 0, 1, 0, 1]


In [36]:
#A and B are the spin up and spin down tensor networks
def net_2x2(A, B):
    AB=[A,B]
    C = [0 for i in range (2**4)]
    count=0
    for i ,j ,k ,l in indices(4,2):  
        C[count]=torch.einsum('ijab, jicd, klba, lkdc', AB[i][0][0], AB[j][0][1], AB[k][1][0], AB[l][1][1])
        count+=1
    return C

# A = [[0 for i in range(2)]]*2
# B = [[0 for i in range(2)]]*2

# for i in range (2):
#     for j in range(2):
#         A[i][j]=torch.rand((2,2,2,2))
#         B[i][j]=torch.rand((2,2,2,2))
# print(net_2x2(A,B))


'"\nA = [[0 for i in range(2)]]*2\nB = [[0 for i in range(2)]]*2\n\nfor i in range (2):\n    for j in range(2):\n        A[i][j]=torch.rand((2,2,2,2))\n        B[i][j]=torch.rand((2,2,2,2))\nprint(net_2x2(A,B))\n'

In [37]:
def net_3x3(A,B):
    AB=[A,B]
    C = [0 for i in range (2**9)]
    count=0
    for i ,j ,k ,l ,m ,n ,o ,p ,q in indices(9, 2):    
        C[count]=torch.einsum('abjk, camn, bcpq, delj, fdom, efrp, ghkl, igno, hiqr', AB[i][0][0], AB[j][0][1], AB[k][0][2], AB[l][1][0], AB[m][1][1], AB[n][1][2], AB[o][2][0], AB[p][2][1], AB[q][2][2])
        count+=1
    return C


# A = [[0 for i in range(3)]]*3
# B = [[0 for i in range(3)]]*3
        
# for i in range (3):
#     for j in range(3):
#         A[i][j]=torch.rand((2,2,2,2))
#         B[i][j]=torch.rand((2,2,2,2))
# print(net_3x3(A,B))


'\nA = [[0 for i in range(3)]]*3\nB = [[0 for i in range(3)]]*3\n        \nfor i in range (3):\n    for j in range(3):\n        A[i][j]=torch.rand((2,2,2,2))\n        B[i][j]=torch.rand((2,2,2,2))\nprint(net_3x3(A,B))\n'

In [48]:
def net_4x4(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(4, samples)
    for i in (indis):
        CI = instant_indices(16,i) #current index
        X = torch.einsum('abeh, cail, dcmp, bdqt-> ehilmpqt', AB[CI[0]][0][0], AB[CI[1]][0][1], AB[CI[2]][0][2], AB[CI[3]][0][3])
        Y = torch.einsum('abfe, caji, dcnm, bdrq-> fejinmrq', AB[CI[4]][1][0], AB[CI[5]][1][1], AB[CI[6]][1][2], AB[CI[7]][1][3])
        Z = torch.einsum('abgf, cakj, dcon, bdsr-> gfkjonsr', AB[CI[8]][2][0], AB[CI[9]][2][1], AB[CI[10]][2][2], AB[CI[11]][2][3])
        W = torch.einsum('abhg, calk, dcpo, bdts-> hglkpots', AB[CI[12]][3][0], AB[CI[13]][3][1], AB[CI[13]][3][2], AB[CI[13]][3][3])
        C[count] = torch.einsum('ehilmpqt, fejinmrq, gfkjonsr, hglkpots', X, Y, Z, W)
        count+=1
    return C

# A = [[0 for i in range(4)]]*4
# B = [[0 for i in range(4)]]*4
        
# for i in range (4):
#     for j in range(4):
#         A[i][j]=torch.rand((2,2,2,2))
#         B[i][j]=torch.rand((2,2,2,2))
# print(net_4x4(A,B,100))

In [54]:
def net_5x5(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(5, samples)
    for i in (indis):
        CI = instant_indices(25,i) #current index
        X = torch.einsum('abfg, cahi, dcjk, edlm, beno -> fghijklmno', AB[CI[0]][0][0], AB[CI[1]][0][1], AB[CI[2]][0][2], AB[CI[3]][0][3], AB[CI[4]][0][4]) 
        Y = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[CI[5]][1][0], AB[CI[6]][1][1], AB[CI[7]][1][2], AB[CI[8]][1][3], AB[CI[9]][1][4])
        XY = torch.einsum('fghijklmno, pfqhrjsltn -> gpiqkrmson', X, Y)
        Z = torch.einsum('abfp, cahq, dcjr, edls, bent -> fphqjrlsnt' , AB[CI[10]][2][0], AB[CI[11]][2][1], AB[CI[12]][2][2], AB[CI[13]][2][3], AB[CI[14]][2][4])
        XYZ = torch.einsum('gpiqkrmson, fphqjrlsnt -> gfihkjmlon', XY, Z)
        U = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[CI[15]][3][0], AB[CI[16]][3][1], AB[CI[17]][3][2], AB[CI[18]][3][3], AB[CI[19]][3][4])
        XYZU = torch.einsum('gfihkjmlon, pfqhrjsltn -> gpiqkrmsot', XYZ, U)
        V = torch.einsum('abgp, caiq, dckr, edms, beot -> gpiqkrmsot', AB[CI[20]][4][0], AB[CI[21]][4][1], AB[CI[22]][4][2], AB[CI[23]][4][3], AB[CI[24]][4][4])
        C[count] = torch.einsum('gpiqkrmsot, gpiqkrmsot', XYZU, V)
        count+=1
    return C    

# A = [[0 for i in range(5)]]*5
# B = [[0 for i in range(5)]]*5
        
# for i in range (5):
#     for j in range(5):
#         A[i][j]=torch.rand((2,2,2,2))
#         B[i][j]=torch.rand((2,2,2,2))
# print(net_5x5(A,B,100))
    

In [56]:
def net_6x6(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(6, samples)
    for i in (indis):
        CI = instant_indices(36,i) #current index
        X = torch.einsum('abnh, caoi, dcpj, edqk, ferl, bfsm -> nhoipjqkrlsm', AB[CI[0]][0][0], AB[CI[1]][0][1], AB[CI[2]][0][2], AB[CI[3]][0][3], AB[CI[4]][0][4], AB[CI[5]][0][5]) 
        Y = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[CI[6]][1][0], AB[CI[7]][1][1], AB[CI[8]][1][2], AB[CI[9]][1][3], AB[CI[10]][1][4], AB[CI[11]][1][5])
        XY = torch.einsum('nhoipjqkrlsm, tnuovpwqxrys -> htiujvkwlxmy', X, Y)
        Z = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[CI[12]][2][0], AB[CI[13]][2][1], AB[CI[14]][2][2], AB[CI[15]][2][3], AB[CI[16]][2][4], AB[CI[17]][2][5])
        XYZ = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XY, Z)
        U = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[CI[18]][3][0], AB[CI[19]][3][1], AB[CI[20]][3][2], AB[CI[21]][3][3], AB[CI[22]][3][4], AB[CI[23]][3][5])
        XYZU = torch.einsum('hniojpkqlrms, tnuovpwqxrys -> htiujvkwlxmy', XYZ, U)
        V = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[CI[24]][4][0], AB[CI[25]][4][1], AB[CI[26]][4][2], AB[CI[27]][4][3], AB[CI[28]][4][4], AB[CI[29]][4][5])
        XYZUV = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XYZU, V)
        W = torch.einsum('abhn, caio, dcjp, edkq, felr, bfms -> hniojpkqlrms', AB[CI[30]][5][0], AB[CI[31]][5][1], AB[CI[32]][5][2], AB[CI[33]][5][3], AB[CI[34]][5][4], AB[CI[35]][5][5])
        C[count] = torch.einsum('hniojpkqlrms, hniojpkqlrms', XYZUV, W)
        count+=1
    return C

# A = [[0 for i in range(6)]]*6
# B = [[0 for i in range(6)]]*6
        
# for i in range (6):
#     for j in range(6):
#         A[i][j]=torch.rand((2,2,2,2))
#         B[i][j]=torch.rand((2,2,2,2))
# print(net_6x6(A,B,100))